In [1]:
from splinter import Browser 
from bs4 import BeautifulSoup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pymongo
from pymongo import MongoClient
from flask import Flask, render_template

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\nadzema\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
Mars_nasa_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(Mars_nasa_url)

In [4]:
#create HTML object
html = browser.html
#parse HTML with BEAUTIFULSOUP
soup = BeautifulSoup(html, 'html.parser')

In [5]:
#visit the website NASA url
first_item = soup.find('li', class_='slide')

#
title = first_item.find('div', class_='content_title').text
print(title)
print(first_item)

NASA's Curiosity Mars Rover Takes Selfie With 'Mont Mercou'
<li class="slide"><div class="image_and_description_container"><a href="/news/8901/nasas-curiosity-mars-rover-takes-selfie-with-mont-mercou/" target="_self"><div class="rollover_description"><div class="rollover_description_inner">The rover also snapped a pair of panoramas to create a 3D view of the stark cliff face featured in the selfie.</div><div class="overlay_arrow"><img alt="More" src="/assets/overlay-arrow.png"/></div></div><div class="list_image"><img alt="NASA’s Curiosity Mars rover used two cameras to create this selfie in front of Mont Mercou, a rock outcrop that stands 20 feet (6 meters) tall." src="/system/news_items/list_view_images/8901_1-PIA24543-Curiosity's-Selfie-at-Mont-Mercou-main-320x240.jpg"/></div><div class="bottom_gradient"><div><h3>NASA's Curiosity Mars Rover Takes Selfie With 'Mont Mercou'</h3></div></div></a><div class="list_text"><div class="list_date">March 30, 2021</div><div class="content_title"

In [6]:
news_p = first_item.find('div', class_='article_teaser_body').text
print(title)
print (news_p)

NASA's Curiosity Mars Rover Takes Selfie With 'Mont Mercou'
The rover also snapped a pair of panoramas to create a 3D view of the stark cliff face featured in the selfie.


In [7]:
# Setup splinter again for a different url
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\nadzema\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [8]:
images_url = 'https://www.jpl.nasa.gov/images?search=&category=Mars'
browser.visit(images_url)

In [9]:
html2 = browser.html
soup2 = BeautifulSoup(html2, 'html.parser')

In [10]:
#visit the url  
featured_image_url = soup2.find('img', class_= 'BaseImage')['src']
print(featured_image_url)

https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24513.2e16d0ba.fill-400x400-c50.jpg


In [11]:
#data on Mars URL
mars_facts_url = 'https://space-facts.com/mars/'

In [12]:
mars_facts_html = pd.read_html(mars_facts_url)

#html format data 1st index compares Mars to Earth
mars_earth_df = mars_facts_html[1]

#Only Mars data 
mars_df = mars_earth_df.drop(columns=['Earth']).rename(columns={'Mars - Earth Comparison': 'Description'})
mars_df = mars_df.set_index('Description')
mars_df

,Mars
Description,
Diameter:,"6,779 km"
Mass:,6.39 × 10^23 kg
Moons:,2
Distance from Sun:,"227,943,824 km"
Length of Year:,687 Earth days
Temperature:,-87 to -5 °C


In [13]:
#Saving mars dataframe as html file
mars_df.to_html('mars_facts.html')

In [14]:
#convert mars dataframe to html string
mars_facts_string = mars_df.to_html (header=True, index=True)
print (mars_facts_string)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Mars</th>
    </tr>
    <tr>
      <th>Description</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Diameter:</th>
      <td>6,779 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2</td>
    </tr>
    <tr>
      <th>Distance from Sun:</th>
      <td>227,943,824 km</td>
    </tr>
    <tr>
      <th>Length of Year:</th>
      <td>687 Earth days</td>
    </tr>
    <tr>
      <th>Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
  </tbody>
</table>


In [15]:
#Mars Hemispheres
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres_url)

In [16]:
html4 = browser.html
soup4 = BeautifulSoup(html4, 'html.parser')

In [17]:
#all urls for 4 hemispheres of Mars
hemosphere_urls = soup4.find_all('div', class_='description')

In [18]:
main_url = 'https://astrogeology.usgs.gov'
hemisphere_image_urls = []
for item in hemosphere_urls:
    each_url = item.find('a')['href']
    full_url = main_url + each_url
    title = item.find('h3').text
    
    hemisphere_image_urls.append({'Title': title, 'Hemisphere Url':full_url})

In [27]:
print(hemisphere_image_urls)

[{'Title': 'Cerberus Hemisphere Enhanced', 'Hemisphere Url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'}, {'Title': 'Schiaparelli Hemisphere Enhanced', 'Hemisphere Url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced'}, {'Title': 'Syrtis Major Hemisphere Enhanced', 'Hemisphere Url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced'}, {'Title': 'Valles Marineris Hemisphere Enhanced', 'Hemisphere Url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced'}]


In [30]:
all_data_mars = {
'Title' : title,
'Mars_Paragraph': news_p,
'Featured_image': featured_image_url,
'Mars_Facts_Table': mars_facts_string,
'Hemisphere_image_urls': hemisphere_image_urls,
}

In [31]:
print (all_data_mars)

{'Title': 'Valles Marineris Hemisphere Enhanced', 'Mars_Paragraph': 'The rover also snapped a pair of panoramas to create a 3D view of the stark cliff face featured in the selfie.', 'Featured_image': 'https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24513.2e16d0ba.fill-400x400-c50.jpg', 'Mars_Facts_Table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n  </t

In [37]:
#Connecting to PyMongo localhost & assigning variables
client = pymongo.MongoClient('mongodb://localhost:27017')
db = client.mars_db
collection = db.mars 

collection.insert(all_data_mars)